# EarthReach - Weather Chart Description Generation Example 

### Introduction

EarthReach is a Python library for generating natural language descriptions of meteorological data visualizations. The library extends **earthkit-plots** by providing automated text generation capabilities for weather charts, enabling programmatic conversion of visual data representations into structured textual descriptions.

The system implements a dual-LLM architecture consisting of a generator agent and an evaluator agent. The generator creates initial descriptions from chart images and associated GRIB file metadata, while the evaluator assesses output quality across multiple criteria including scientific accuracy, coherence, and meteorological relevance. This iterative process continues until quality thresholds are met or maximum iterations are reached.

This notebook demonstrates the workflow for using the EarthReachAgent API to generate descriptions of weather charts.

### Prerequisites

Before running this notebook, ensure you have:

- **Python 3.12+** with all project dependencies installed
- **[Climate Data Store](https://cds.climate.copernicus.eu/how-to-api) API key** configured for accessing meteorological data
- **LLM provider API key** set as an environment variable (supports OpenAI, Anthropic, or compatible endpoints)

For more detailed setup instructions, refer to the [installation documentation](../docs/source/installation.rst).

In [ ]:
import earthkit.plots as ekp
import earthkit.data as ekd
from earth_reach import EarthReachAgent
from textwrap import fill
import warnings
import logging

logging.getLogger('htpx').setLevel(logging.WARNING)
logging.getLogger('google_genai').setLevel(logging.WARNING)

warnings.filterwarnings('ignore')

In [ ]:
# Load your data with earthkit-data
request = {
    "product_type": ["reanalysis"],
    "variable": ["2m_temperature", "mean_sea_level_pressure"],
    "day": ['30'],
    "month": ['04'],
    "year": ['2025'],
    "time": ['12:00'],
    "data_format": "grib",
    "download_format": "unarchived",
}
dataset = "reanalysis-era5-single-levels"
data = ekd.from_source("cds", dataset, request)

# Preprocess the data as you like, but make sure these two variables are present in the dataset
sub_data = data.sel(param=["2t", "msl"], typeOfLevel="surface")

# Create a weather chart with earthkit-plots
figure = ekp.quickplot(sub_data, domain='France', units=['celsius', 'hPa'], mode="overlay")

# Generate description
agent = EarthReachAgent(provider="gemini", model_name='gemini-2.5-pr') 
description = agent.generate_alt_description(figure, sub_data)

# Visualize description
print('\nGenerated Description:')
print('-' * 22)
print(fill(description, 88))